In [25]:
import requests
import pandas as pd
import numpy as np
import time
import sqlite3
from datetime import datetime,timedelta,date
import time
from pandas.tseries.offsets import BDay

In [26]:
con = sqlite3.connect("stock.db")

In [27]:
def calBuyDays(buy,pre_day):
    new_day = 0
    if pre_day >= 0:
        if buy > 0:
            new_day = pre_day + 1
        elif  buy < 0:
            new_day = -1
        elif buy == 0:
             new_day = 0
    elif pre_day < 0:      
        if buy > 0:
            new_day = 1        
        elif  buy < 0:
            new_day = pre_day - 1
        elif buy == 0:
            new_day = 0
    else:
         new_day = None
            
    #print('new_day',new_day)
    return new_day
    

In [28]:
weekmask = [1,1,1,1,1,0,0]
holidays = ['2018-04-04','2018-04-05','2018-05-01','2018-06-18','2018-09-24','2018-10-10']
dateformat1 = '%Y%m%d'
dateformat2 = '%Y-%m-%d'

In [29]:
searchdate = pd.date_range(start='2018-10-01', periods=5)
for d in searchdate:
    print('== start',datetime.now())
    d1 = datetime.strftime(d,dateformat1)
    d2 = datetime.strftime(d,dateformat2)    
    pre_date = np.busday_offset(d, -1, roll='forward',weekmask=weekmask,holidays=holidays)
    pre_date = pre_date.astype(datetime)
    pre_datef = pre_date.strftime(dateformat2)
    print('d1',d1)
    print('d2',d2)
    print('pre_datef',pre_datef)
    
    urlDailyVol = "http://www.twse.com.tw/fund/T86?response=json&date={}&selectType=ALLBUT0999".format(d1)
    html = requests.get(urlDailyVol)
    html.encoding = "utf-8"
    dataInfo = html.json()

    if dataInfo['stat'] == "很抱歉，沒有符合條件的資料!" :
        print('--off--')
        continue
    
    select = "select 交易日,證券代號,證券名稱,外資連N日買超,投信連N日買超,自營商連N日買超 from investor where 交易日 = '{}'".format(pre_datef)
    df_pre = pd.read_sql(select,con)

    orglist = dataInfo["data"]
    dfall0 = pd.DataFrame.from_records(orglist,columns=dataInfo['fields'])
    dfall = dfall0[['證券代號','證券名稱','外陸資買賣超股數(不含外資自營商)','投信買賣超股數','自營商買賣超股數']]
    
    dfall['外資買賣超張數'] = round(dfall['外陸資買賣超股數(不含外資自營商)'].str.replace(',','').astype('int')/1000)
    dfall['投信買賣超張數'] = round(dfall['投信買賣超股數'].str.replace(',','').astype('int')/1000)
    dfall['自營商買賣超張數'] = round(dfall['自營商買賣超股數'].str.replace(',','').astype('int')/1000)
    dfall['外資買超排行'] = dfall['外資買賣超張數'].rank(ascending=False).astype('int')
    dfall['外資賣超排行'] = dfall['外資買賣超張數'].rank(ascending=True).astype('int')
    dfall['投信買超排行'] = dfall['投信買賣超張數'].rank(ascending=False).astype('int')
    dfall['投信賣超排行'] = dfall['投信買賣超張數'].rank(ascending=True).astype('int')
    dfall['自營商買超排行'] = dfall['自營商買賣超張數'].rank(ascending=False).astype('int')
    dfall['自營商賣超排行'] = dfall['自營商買賣超張數'].rank(ascending=True).astype('int')
    dfall['外資連N日買超'] = ''
    dfall['投信連N日買超'] = ''
    dfall['自營商連N日買超'] = ''
    dfall['上市上櫃'] = 1
    dfall.insert(0, '交易日', d2)
    
    for index,row in dfall.iterrows():
        code = row['證券代號']
        buy1 = row['外資買賣超張數']
        buy2 = row['投信買賣超張數']
        buy3 = row['自營商買賣超張數']
        #print('--'*20)
        #print('code=',code)
        #print('buy1=',buy1)
        #print('buy2=',buy2)
        #print('buy3=',buy3)

        x = df_pre.loc[df_pre['證券代號'] == code]
        if x.empty:
            pre_day1 = 0
            pre_day2 = 0
            pre_day3 = 0
            #continue
        else:
            pre_day1 = x['外資連N日買超'].iloc[0]
            if pre_day1 == "":
                pre_day1 = 0

            pre_day2 = x['投信連N日買超'].iloc[0]
            if pre_day2 == "":
                pre_day2 = 0

            pre_day3 = x['自營商連N日買超'].iloc[0]
            if pre_day3 == "":
                pre_day3 = 0
            #print('pre_day1',pre_day1)
            #print('pre_day2',pre_day2)
            #print('pre_day3',pre_day3)

        new_day1 = calBuyDays(buy1,pre_day1)
        new_day2 = calBuyDays(buy2,pre_day2)
        new_day3 = calBuyDays(buy3,pre_day3)
        dfall.loc[index,'外資連N日買超'] = new_day1
        dfall.loc[index,'投信連N日買超'] = new_day2
        dfall.loc[index,'自營商連N日買超'] = new_day3
        
    dfall = dfall[['交易日','證券代號','證券名稱','外資買賣超張數','外資連N日買超','外資買超排行','外資賣超排行'
                   ,'投信買賣超張數','投信連N日買超','投信買超排行','投信賣超排行'
                   ,'自營商買賣超張數','自營商連N日買超','自營商買超排行','自營商賣超排行','上市上櫃']]
    dfall.to_sql('investor',con,if_exists='append',index =False)
    print(d2,' -- save ok')
    
    print('== end',datetime.now())
    print('Ok')

== start 2018-10-08 14:40:26.765567
d1 20181001
d2 2018-10-01
pre_datef 2018-09-28


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

2018-10-01  -- save ok
== end 2018-10-08 14:40:34.876030
Ok
== start 2018-10-08 14:40:34.876030
d1 20181002
d2 2018-10-02
pre_datef 2018-10-01
2018-10-02  -- save ok
== end 2018-10-08 14:40:43.353515
Ok
== start 2018-10-08 14:40:43.353515
d1 20181003
d2 2018-10-03
pre_datef 2018-10-02
2018-10-03  -- save ok
== end 2018-10-08 14:40:52.141018
Ok
== start 2018-10-08 14:40:52.142018
d1 20181004
d2 2018-10-04
pre_datef 2018-10-03
2018-10-04  -- save ok
== end 2018-10-08 14:41:00.395490
Ok
== start 2018-10-08 14:41:00.395490
d1 20181005
d2 2018-10-05
pre_datef 2018-10-04
2018-10-05  -- save ok
== end 2018-10-08 14:41:08.612960
Ok


In [30]:
con.commit()
print('== commit ok ==')

== commit ok ==


In [31]:
con.close()
print('== close ==')

== close ==


In [32]:
#df = pd.read_excel("c:/pythondata/2018-06-08法人.xlsx",sheet_name='2018-04-02')
#con = sqlite3.connect("stock.db")
#df.to_sql('investor',con,if_exists='append',index =False)
#print('save ok')